<a href="https://colab.research.google.com/github/Vaishnavi1657/23CSBTB27-28/blob/main/Game_Strategy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Implementing Game Strategy


##PART-1

In [ ]:
from collections import namedtuple,Counter,defaultdict
import random
import math
import functools
cache=functools.lru_cache(10**6)
class Game:
  def actions(self, state):
    raise NotImplementedError
  def result(self, state, move):
    raise NotImplementedError
  def is_terminal(self, state):
    return not self.actions(state)
  def utility(self, state, player):
    raise NotImplementedError
  def play_game(game, strategies: dict, verbose=False):
    state = game.initial
    while not game.is_terminal(state):
      player = state.to_move
      move = strategies[player](game, state)
      state = game.result(state, move)
    if verbose:
      print( ' Player ', player,' move:', move)
      print(state)
    return state

##PART-2

In [ ]:
def play_game(game, strategies: dict, verbose=False):
    """
    Play a turn-taking game. 'strategies' is a {player_name: function} dict,
    where function(state, game) is used to get the player's move.
    """
    state = game.initial
    while not game.is_terminal(state):
        player = state.to_move
        move = strategies[player](game, state)
        state = game.result(state, move)
        if verbose:
            print('Player', player, 'move:', move)
            print(state)
        return state

##PART-3

In [1]:
import math

def alphabeta_search(game, state):
    """
    Search game to determine the best action; use alpha-beta pruning.
    Search all the way to the leaves.
    """
    player = state.to_move

    def max_value(state, alpha, beta):
      if game.is_terminal(state):
        return game.utility(state, player), None
        v, move = -math.inf, None
        for a in game.actions(state):
            v2, _ = min_value(game.result(state, a), alpha, beta)
            if v2 > v:
                v, move = v2, a
            alpha = max(alpha, v)
            if v >= beta:
                return v, move
        return v, move

    def min_value(state, alpha, beta):
        if game.is_terminal(state):
            return game.utility(state, player), None
        v, move = math.inf, None
        for a in game.actions(state):
            v2, _ = max_value(game.result(state, a), alpha, beta)
            if v2 < v:
                v, move = v2, a
            beta = min(beta, v)
            if v <= alpha:
                return v, move
        return v, move

    return max_value(state, -math.inf, math.inf)

##PART-4

In [2]:
import math

def alphabeta_search(game, state):
    """
    Search game to determine the best action; use alpha-beta pruning.
    Search all the way to the leaves.
    """
    player = state.to_move

    def max_value(state, alpha, beta):
        if game.is_terminal(state):
            return game.utility(state, player), None
        v, move = -math.inf, None
        for a in game.actions(state):
            v2, _ = min_value(game.result(state, a), alpha, beta)
            if v2 > v:
                v, move = v2, a
            alpha = max(alpha, v)
            if v >= beta:
                return v, move
        return v, move

    def min_value(state, alpha, beta):
        if game.is_terminal(state):
            return game.utility(state, player), None
        v, move = math.inf, None
        for a in game.actions(state):
            v2, _ = max_value(game.result(state, a), alpha, beta)
            if v2 < v:
                v, move = v2, a
            beta = min(beta, v)
            if v <= alpha:
                return v, move
        return v, move

    return max_value(state, -math.inf, math.inf)